## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-24-06-51-00 +0000,wsj,NatWest Lifts Guidance Again After Strong Cust...,https://www.wsj.com/business/earnings/natwest-...
1,2025-10-24-06-49-48 +0000,bbc,Chris Mason: Extraordinary Caerphilly by-elect...,https://www.bbc.com/news/articles/c4gj48q4x39o...
2,2025-10-24-06-46-35 +0000,nypost,California son found guilty in brutal 2019 sla...,https://nypost.com/2025/10/24/us-news/californ...
3,2025-10-24-06-43-26 +0000,bbc,Monthly retail sales hit highest level since 2022,https://www.bbc.com/news/articles/cdxrnz35k1no...
4,2025-10-24-06-42-00 +0000,wsj,UBS Names New Chief Operating Officer in Execu...,https://www.wsj.com/business/c-suite/ubs-names...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
65,trump,67
38,new,20
346,sanctions,14
102,off,14
347,russian,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
314,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...,134
168,2025-10-23-20-25-31 +0000,nyt,Trump’s Sanctions on Russian Oil Sector Ratche...,https://www.nytimes.com/2025/10/23/us/politics...,130
293,2025-10-23-13-32-00 +0000,nyt,Trump Imposes Sanctions on Russian Oil Compani...,https://www.nytimes.com/2025/10/22/us/politics...,125
198,2025-10-23-18-09-53 +0000,nypost,"China, India curb Russian oil purchases after ...",https://nypost.com/2025/10/23/world-news/china...,125
45,2025-10-24-03-21-49 +0000,cbc,Trump says he's terminating trade talks with C...,https://www.cbc.ca/news/world/trump-trade-nego...,124


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
314,134,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...
144,56,2025-10-23-21-24-00 +0000,wsj,Lawmakers failed to provide a last-minute repr...,https://www.wsj.com/politics/policy/air-traffi...
48,54,2025-10-24-03-00-00 +0000,wsj,Will India and China Be Able to Resist U.S. Sa...,https://www.wsj.com/world/asia/india-china-tru...
59,45,2025-10-24-01-40-54 +0000,nypost,White House official suggests married ex-GOP R...,https://nypost.com/2025/10/23/us-news/white-ho...
12,45,2025-10-24-06-05-22 +0000,nypost,DOJ tells California officials to back off ‘ap...,https://nypost.com/2025/10/24/us-news/doj-tell...
291,42,2025-10-23-13-34-07 +0000,nypost,"King Charles, Pope Leo pray together in rare m...",https://nypost.com/2025/10/23/world-news/king-...
46,42,2025-10-24-03-16-11 +0000,nypost,Trump terminates trade talks with Canada over ...,https://nypost.com/2025/10/23/us-news/trump-te...
249,41,2025-10-23-15-38-53 +0000,nypost,Trump pardons convicted Binance founder Changp...,https://nypost.com/2025/10/23/business/trump-p...
205,41,2025-10-23-18-06-16 +0000,nypost,French museum was robbed of 2K gold and silver...,https://nypost.com/2025/10/23/world-news/a-sec...
276,37,2025-10-23-14-38-47 +0000,nypost,"Trump says ‘I will’ visit Gaza, calls ‘respect...",https://nypost.com/2025/10/23/us-news/trump-sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
